In [1]:
#Import Required libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
#Import Dependencies for sentiment analysis using our neural network
import tensorflow as tf
import numpy
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import alpaca_trade_api as tradeapi

In [2]:
finviz_url = "https://finviz.com/quote.ashx?t="
ticker = "TSLA"
url = finviz_url + ticker

In [4]:
req = Request(url=url, headers={'user-agent': 'my-app'})
response = urlopen(req)

news_table = {}
html = BeautifulSoup(response)
news_table = html.find(id='news-table')
# news_table

dataRows = news_table.findAll('tr')



In [5]:
df = pd.DataFrame(columns=['News_Title', 'Time'])
for i, table_row in enumerate(dataRows):
    a_text = table_row.a.text
    td_text = table_row.td.text
    
    df = df.append({'News_Title': a_text, 'Time': td_text}, ignore_index=True)



In [6]:
with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
model = tf.keras.models.load_model('./model1.h5')

In [7]:
oov_tok = '<OOV>'
trunc_type = 'post'
padding_type='post'
vocab_size =1000
max_length = 142

def preprocessText(text):
    sequences = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type,
                          truncating=trunc_type)
    return padded

prep = preprocessText(df.News_Title)
prep = model.predict(prep)
df['sent'] = np.argmax(prep, axis=-1)



In [9]:
api = tradeapi.REST("PKAFHJ9U5CUF1FD3QDS7", "QeBhKnBE7nbpJZZmzeeJ347JqiUQ0RwyMaqxlIxE", "https://paper-api.alpaca.markets")
account = api.get_account()
# print(account)

In [13]:
modeSentiment = df.sent.mode().iloc[0]
# print(modeSentiment)

if modeSentiment == 1:
    print("Neutral Sentiment, Nothing to do!")
elif modeSentiment == 0:
    api.submit_order(
        symbol=ticker,
        qty=1,
        side='sell',
        type='market',
        time_in_force='gtc'
    )
    print("SELL")
elif modeSentiment == 2:
    api.submit_order(
        symbol=ticker,
        qty=1,
        side='buy',
        type='market',
        time_in_force='gtc'
    )
    print("BUY")

Neutral Sentiment, Nothing to do!


In [ ]:
# you can create your own data set to see changes.